In [ ]:
from panzer.request import get, post
from panzer.limits import BinanceRateLimiter

In [ ]:
def test_rate_limiter(rate_limiter):

    BASE_URL = 'https://api.binance.com'

    for endpoint, weight, params, signed, method in CALLS:
        url = BASE_URL + endpoint
        is_order = url.endswith("/api/v3/order")
        can = rate_limiter.can_make_request(weight=weight, is_order=is_order)
        print(f"\n----------------\n"
              f"Can make request to {endpoint} w={weight} is_order={is_order} can_make={can}\n{rate_limiter.get()}\n")

        # Make the API call
        if can:
            if method == "GET":
                response, headers = get(url, params=params, full_sign=signed)
            elif method == "POST":
                response, headers = post(url, params=params, full_sign=signed)
            else:
                raise ValueError(f"Unsupported method: {method}")
            x_headers = {h: v for h, v in headers.items() if h.startswith('x-mbx-')}
            print(f"Response Headers for {endpoint}: {x_headers}")
            rate_limiter.update_from_headers(headers)
        else:
            raise Exception(f"Cannot make request to {endpoint}. Rate limit exceeded.")
    # loop for overcharging
    test_rate_limiter(rate_limiter)

In [ ]:
# Define base URL for Binance API

# Define public endpoints for testing
CALLS = [
    ('/api/v3/time', 1, [], False, "GET"),
    ('/api/v3/exchangeInfo', 20, [], False, "GET"),
    ('/api/v3/ticker/price', 4, [], False, "GET"),
    # ('/api/v3/order/test', 1, {'symbol': "BTCUSDT",
    #                            'side': "SELL",
    #                            'type': "LIMIT",
    #                            'timeInForce': 'GTC',  # Good 'Til Canceled
    #                            'quantity': 0.001,
    #                            'price': 80000,
    #                            'recvWindow': 10000,
    #                            'timestamp': int(time() * 1000)},
    #  True, "POST"),
    # ('/api/v3/order', 1, {'symbol': "BTCUSDT",
    #                       'side': "SELL",
    #                       'type': "LIMIT",
    #                       'timeInForce': 'GTC',  # Good 'Til Canceled
    #                       'quantity': 0.001,
    #                       'price': 80000,
    #                       'recvWindow': 5000,
    #                       'timestamp': int(time() * 1000)},
    #  True, "POST"),
    # ('/api/v3/order', 1, {'symbol': "BTCUSDT",
    #                       'side': "SELL",
    #                       'type': "LIMIT",
    #                       'timeInForce': 'GTC',  # Good 'Til Canceled
    #                       'quantity': 0.001,
    #                       'price': 80000,
    #                       'recvWindow': 5000,
    #                       'timestamp': int(time() * 1000)},
    #  True, "POST"),
    # ('/api/v3/order', 1, {'symbol': "BTCUSDT",
    #                       'side': "SELL",
    #                       'type': "LIMIT",
    #                       'timeInForce': 'GTC',  # Good 'Til Canceled
    #                       'quantity': 0.001,
    #                       'price': 80000,
    #                       'recvWindow': 5000,
    #                       'timestamp': int(time() * 1000)},
    #  True, "POST"),
]

In [ ]:
rate_limiter = BinanceRateLimiter(info_level="DEBUG")

In [ ]:
rate_limiter

In [ ]:
test_rate_limiter(rate_limiter)